In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/brazilian-ecommerce/olist_customers_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_sellers_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_order_reviews_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_order_items_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_products_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_geolocation_dataset.csv
/kaggle/input/brazilian-ecommerce/product_category_name_translation.csv
/kaggle/input/brazilian-ecommerce/olist_orders_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_order_payments_dataset.csv


In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from math import radians, cos, sin, asin, sqrt

In [3]:
# root path for all dataset
root_path = '../input/brazilian-ecommerce/'

# olist_customers_dataset
customers = pd.read_csv(root_path + "olist_customers_dataset.csv")
customers.head(3)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP


In [4]:
customers = customers.drop(columns = ['customer_zip_code_prefix'])
customers.head()

,customer_id,customer_unique_id,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,campinas,SP


In [5]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_id         99441 non-null  object
 1   customer_unique_id  99441 non-null  object
 2   customer_city       99441 non-null  object
 3   customer_state      99441 non-null  object
dtypes: object(4)
memory usage: 3.0+ MB


In [6]:
customers.shape

(99441, 4)

In [7]:
customers = customers.drop_duplicates()
customers.shape

(99441, 4)

In [8]:
#fréquences de valeurs manquantes
freq = (customers.isna().sum() / len(customers)) * 100
freq

customer_id           0.0
customer_unique_id    0.0
customer_city         0.0
customer_state        0.0
dtype: float64

In [9]:
# Supprimer les doublons
customers = customers.drop_duplicates()

In [10]:
customers.head()

,customer_id,customer_unique_id,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,campinas,SP


In [11]:
# Enregistrer le DataFrame en tant que fichier CSV dans le répertoire /kaggle/working/
output_path = '/kaggle/working/customers_cleaning.csv'
customers.to_csv(output_path, index=False)

# Afficher l'emplacement du fichier
print(f"Le fichier a été enregistré à l'emplacement : {output_path}")

Le fichier a été enregistré à l'emplacement : /kaggle/working/customers_cleaning.csv


In [12]:
# olist_orders_dataset
orders = pd.read_csv(root_path + "olist_orders_dataset.csv")
orders.head(3)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00


In [13]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [14]:
# Supprimer les colonnes inutiles
orders = orders.drop(columns=['order_approved_at', 'order_delivered_carrier_date'])
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-16 18:17:02,2018-02-26 00:00:00


In [15]:
# Vérifier les colonnes disponibles dans le DataFrame
print(orders.columns)

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_delivered_customer_date', 'order_estimated_delivery_date'],
      dtype='object')


In [16]:
orders = orders.drop_duplicates()
orders.shape

(99441, 6)

In [17]:
#fréquences de valeurs manquantes
freq = (orders.isna().sum() / len(orders)) * 100
freq

order_id                         0.000000
customer_id                      0.000000
order_status                     0.000000
order_purchase_timestamp         0.000000
order_delivered_customer_date    2.981668
order_estimated_delivery_date    0.000000
dtype: float64

In [18]:
# Supprimer les lignes avec des valeurs nulles
orders = orders.dropna()

In [19]:
#fréquences de valeurs manquantes
freq = (orders.isna().sum() / len(orders)) * 100
freq

order_id                         0.0
customer_id                      0.0
order_status                     0.0
order_purchase_timestamp         0.0
order_delivered_customer_date    0.0
order_estimated_delivery_date    0.0
dtype: float64

In [20]:
orders.head

<bound method NDFrame.head of                                order_id                       customer_id  \
0      e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
1      53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef   
2      47770eb9100c2d0c44946d9cf07ec65d  41ce2a54c0b03bf3443c3d931a367089   
3      949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82   
4      ad21c59c0840e6cb83a9ceb5573f8159  8ab97904e6daea8866dbdbc4fb7aad2c   
...                                 ...                               ...   
99436  9c5dedf39a927c1b2549525ed64a053c  39bd1228ee8140590ac3aca26f2dfe00   
99437  63943bddc261676b46f01ca7ac2f7bd8  1fca14ff2861355f6e5f14306ff977a7   
99438  83c1379a015df1e13d02aae0204711ab  1aa71eb042121263aafbe80c1b562c9c   
99439  11c177c8e97725db2631073c19f07b62  b331b74b18dc79bcdf6532d51e1637c1   
99440  66dea50a8b16d9b4dee7af250b4be1a5  edb027a75a1449115f6b43211ae02a24   

      order_status order_purchase_timestamp o

In [21]:
# Enregistrer le DataFrame en tant que fichier CSV dans le répertoire /kaggle/working/
output_path = '/kaggle/working/orders_cleaning.csv'
orders.to_csv(output_path, index=False)

# Afficher l'emplacement du fichier
print(f"Le fichier a été enregistré à l'emplacement : {output_path}")

Le fichier a été enregistré à l'emplacement : /kaggle/working/orders_cleaning.csv


In [22]:
# olist_order_items_dataset
order_items = pd.read_csv(root_path + "olist_order_items_dataset.csv")
order_items.head(3)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.9,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.0,17.87


In [23]:
order_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


In [24]:
# Supprimer les colonnes inutiles
order_items = order_items.drop(columns=['seller_id', 'shipping_limit_date'])
order_items.head()

,order_id,order_item_id,product_id,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,199.90,18.14


In [25]:
order_items = order_items.drop_duplicates()
order_items.shape

(112650, 5)

In [26]:
#fréquences de valeurs manquantes
freq = (order_items.isna().sum() / len(order_items)) * 100
freq

order_id         0.0
order_item_id    0.0
product_id       0.0
price            0.0
freight_value    0.0
dtype: float64

In [27]:
# Enregistrer le DataFrame en tant que fichier CSV dans le répertoire /kaggle/working/
output_path = '/kaggle/working/order_items_cleaning.csv'
order_items.to_csv(output_path, index=False)

# Afficher l'emplacement du fichier
print(f"Le fichier a été enregistré à l'emplacement : {output_path}")

Le fichier a été enregistré à l'emplacement : /kaggle/working/order_items_cleaning.csv


In [28]:
# olist_order_payments_dataset
order_payments = pd.read_csv(root_path + "olist_order_payments_dataset.csv")
order_payments.head(3)

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71


In [29]:
order_payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


In [30]:
order_payments = order_payments.drop_duplicates()
order_payments.shape

(103886, 5)

In [31]:
#fréquences de valeurs manquantes
freq = (order_payments.isna().sum() / len(order_payments)) * 100
freq

order_id                0.0
payment_sequential      0.0
payment_type            0.0
payment_installments    0.0
payment_value           0.0
dtype: float64

In [32]:
# Enregistrer le DataFrame en tant que fichier CSV dans le répertoire /kaggle/working/
output_path = '/kaggle/working/order_payments_cleaning.csv'
order_payments.to_csv(output_path, index=False)

# Afficher l'emplacement du fichier
print(f"Le fichier a été enregistré à l'emplacement : {output_path}")

Le fichier a été enregistré à l'emplacement : /kaggle/working/order_payments_cleaning.csv


In [33]:
# olist_order_reviews_dataset
order_reviews = pd.read_csv(root_path + "olist_order_reviews_dataset.csv")
order_reviews.head(3)

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24


In [34]:
order_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                99224 non-null  object
 1   order_id                 99224 non-null  object
 2   review_score             99224 non-null  int64 
 3   review_comment_title     11568 non-null  object
 4   review_comment_message   40977 non-null  object
 5   review_creation_date     99224 non-null  object
 6   review_answer_timestamp  99224 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


In [35]:
order_reviews = order_reviews.drop(columns=['review_comment_title', 'review_comment_message'])
order_reviews.head()

,review_id,order_id,review_score,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,2018-03-01 00:00:00,2018-03-02 10:26:53


In [36]:
order_reviews = order_reviews.drop_duplicates()
order_reviews.shape

(99224, 5)

In [37]:
#fréquences de valeurs manquantes
freq = (order_reviews.isna().sum() / len(order_reviews)) * 100
freq

review_id                  0.0
order_id                   0.0
review_score               0.0
review_creation_date       0.0
review_answer_timestamp    0.0
dtype: float64

In [38]:
# Enregistrer le DataFrame en tant que fichier CSV dans le répertoire /kaggle/working/
output_path = '/kaggle/working/order_reviews_cleaning.csv'
order_reviews.to_csv(output_path, index=False)

# Afficher l'emplacement du fichier
print(f"Le fichier a été enregistré à l'emplacement : {output_path}")

Le fichier a été enregistré à l'emplacement : /kaggle/working/order_reviews_cleaning.csv


In [39]:
# olist_products_dataset
products = pd.read_csv(root_path + "olist_products_dataset.csv")
products.head(3)

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0


In [40]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB


In [41]:
# Vérifier les colonnes disponibles dans le DataFrame
print(products.columns)

Index(['product_id', 'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm'],
      dtype='object')


In [42]:
# Supprimer les colonnes inutiles
products = products.drop(columns=['product_name_lenght', 'product_description_lenght'])
products.head()

,product_id,product_category_name,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,4.0,625.0,20.0,17.0,13.0


In [43]:
products = products.drop_duplicates()
products.shape

(32951, 7)

In [44]:
#fréquences de valeurs manquantes
freq = (products.isna().sum() / len(products)) * 100
freq

product_id               0.000000
product_category_name    1.851234
product_photos_qty       1.851234
product_weight_g         0.006070
product_length_cm        0.006070
product_height_cm        0.006070
product_width_cm         0.006070
dtype: float64

In [45]:
# Supprimer les lignes avec des valeurs nulles
orders = orders.dropna()

In [46]:
#fréquences de valeurs manquantes
freq = (products.isna().sum() / len(products)) * 100
freq

product_id               0.000000
product_category_name    1.851234
product_photos_qty       1.851234
product_weight_g         0.006070
product_length_cm        0.006070
product_height_cm        0.006070
product_width_cm         0.006070
dtype: float64

In [47]:
# Vérifier les valeurs manquantes dans le DataFrame
missing_values = products.isnull().sum()
print("Valeurs manquantes par colonne :")
print(missing_values)

Valeurs manquantes par colonne :
product_id                 0
product_category_name    610
product_photos_qty       610
product_weight_g           2
product_length_cm          2
product_height_cm          2
product_width_cm           2
dtype: int64


In [48]:
# Supprimer les colonnes avec un pourcentage élevé de valeurs manquantes
threshold = 0.1  # Par exemple, supprimer les colonnes où plus de 10% des valeurs sont manquantes
products = products.loc[:, products.isnull().mean() < threshold]

In [49]:
# Remplir les valeurs manquantes avec la moyenne (pour les colonnes numériques)
products['product_photos_qty'] = products['product_photos_qty'].fillna(products['product_photos_qty'].mean())
products['product_weight_g'] = products['product_weight_g'].fillna(products['product_weight_g'].mean())
products['product_length_cm'] = products['product_length_cm'].fillna(products['product_length_cm'].mean())
products['product_height_cm'] = products['product_height_cm'].fillna(products['product_height_cm'].mean())
products['product_width_cm'] = products['product_width_cm'].fillna(products['product_width_cm'].mean())

# Remplir les valeurs manquantes dans les colonnes catégorielles (par exemple, avec 'inconnu' ou une catégorie par défaut)
products['product_category_name'] = products['product_category_name'].fillna('Inconnu')

In [50]:
#fréquences de valeurs manquantes
freq = (products.isna().sum() / len(products)) * 100
freq

product_id               0.0
product_category_name    0.0
product_photos_qty       0.0
product_weight_g         0.0
product_length_cm        0.0
product_height_cm        0.0
product_width_cm         0.0
dtype: float64

In [51]:
products.head()

,product_id,product_category_name,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,4.0,625.0,20.0,17.0,13.0


In [52]:
# Enregistrer le DataFrame nettoyé dans un fichier CSV
output_path = '/kaggle/working/products_cleaning.csv'
products.to_csv(output_path, index=False)

# Afficher l'emplacement du fichier
print(f"Le fichier a été enregistré à l'emplacement : {output_path}")

Le fichier a été enregistré à l'emplacement : /kaggle/working/products_cleaning.csv


In [53]:
# olist_product_category_name_translation_dataset
categories_en = pd.read_csv(root_path + "product_category_name_translation.csv")
categories_en.head(5)

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


In [54]:
#fréquences de valeurs manquantes
freq = (categories_en.isna().sum() / len(categories_en)) * 100
freq

product_category_name            0.0
product_category_name_english    0.0
dtype: float64

In [55]:
# Enregistrer le DataFrame nettoyé dans un fichier CSV
output_path = '/kaggle/working/categories_en_cleaning.csv'
categories_en.to_csv(output_path, index=False)

# Afficher l'emplacement du fichier
print(f"Le fichier a été enregistré à l'emplacement : {output_path}")

Le fichier a été enregistré à l'emplacement : /kaggle/working/categories_en_cleaning.csv
